In [1]:
from dv import AedatFile
import numpy as np
import os
import re
import torch

In [8]:
dataPath = '../../../DATA/DVS-CIFAR10/processed/aedat4/'
savePath = '../../../DATA/DVS-CIFAR10/processed/tensor_'
files = os.listdir(dataPath)
duration = 5
bins = 40
timewindow = 5*40
label_dict = {'airplane': 0, 'automobile':1, 'bird':2, 'cat':3, 'deer':4, 'dog':5, 'frog':6, 'horse':7, 'ship':8, 'truck':9}

In [9]:
def toTensor(dvs_data, duration, bins):
    
    # Access information of all events by type
    t, x, y, p = dvs_data['timestamp'], dvs_data['x'], dvs_data['y'], dvs_data['polarity']

    tensor_data = torch.zeros([bins, 2, 128, 128], dtype = torch.int)
    for i in range(bins):
        slice_idx = np.where((t<(i+1)*duration) &(t>=i*duration))[0]
        for j in slice_idx:
            tensor_data[i][p[j]][x[j]][y[j]] = 1
    return tensor_data

In [10]:
for file in files:
    with AedatFile(dataPath + file) as f:
        # events will be a named numpy array
        dvs_data = np.hstack([packet for packet in f['events'].numpy()])
        tensor_data = toTensor(dvs_data, duration, bins)
        n_pattern = r'_(\d+)'
        n = int(re.findall(n_pattern, file)[0])
        label_pattern = r'_([A-Za-z]+)_'
        label = int(label_dict[re.findall(label_pattern, file)[0]])
        if n <900:
            torch.save(tensor_data, savePath + 'train/' + str(label) + '_' + str(n) + '.pt')
        else:
            torch.save(tensor_data, savePath + 'test/' + str(label) + '_' + str(n) + '.pt')